In [73]:
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

# Data Loading
df=pd.read_csv('amazon_mobiles_500.csv')
df

,Product Name,MRP,Selling Price,Discount %,Brand,Rating,Review Count,Platform
0,"Samsung Galaxy M36 5G (Velvet Black, 6 GB RAM,...","₹22,999","17,499",(24% off),Samsung,3.8 out of 5 stars,Let us know,Amazon
1,"Samsung Galaxy M06 5G (Sage Green, 4GB RAM, 12...","₹13,999","8,749",(38% off),Samsung,3.8 out of 5 stars,Let us know,Amazon
2,"Samsung Galaxy M05 (Mint Green, 4GB RAM, 64 GB...","₹9,999","6,249",(38% off),Samsung,4.0 out of 5 stars,"5,241",Amazon
3,"Redmi A4 5G (Starry Black, 4GB RAM, 128GB Stor...","₹11,999","8,798",(27% off),Redmi,3.9 out of 5 stars,"5,052",Amazon
4,"POCO M7 5G, Ocean Blue (6GB, 128GB)","₹12,999","9,299",(28% off),POCO,4.0 out of 5 stars,116,Amazon
...,...,...,...,...,...,...,...,...
451,"Samsung Galaxy M36 5G (Orange Haze, 6 GB RAM, ...","₹22,999","17,499",(24% off),Samsung,3.8 out of 5 stars,Let us know,Amazon
452,"Redmi Note 14 5G (Ivy Green, 8GB RAM 128GB Sto...","₹22,999","17,999",(22% off),Redmi,3.8 out of 5 stars,Let us know,Amazon
453,"Lava Bold N1 Pro (Stealth Black, 4 GB RAM, 128...","₹8,399","6,898",(18% off),Lava,4.0 out of 5 stars,Let us know,Amazon
454,"Redmi 13 5G Prime Edition, Hawaiian Blue, 8GB+...","₹19,999","11,699",(42% off),Redmi,4.0 out of 5 stars,Let us know,Amazon


In [75]:
# STEP 1: BASIC CLEANING
# MRP and Selling Price
df['MRP'] = df['MRP'].astype(str).replace('[₹,]', '', regex=True).astype(float)
df['Selling_Price'] = df['Selling Price'].astype(str).replace('[₹,]', '', regex=True).astype(float)
# Discount_Price (MRP - Selling Price)
df['Discount_Price'] = df['MRP'] - df['Selling_Price']
# Ratings
df['Rating'] = df['Rating'].astype(str).str.extract(r'([\d\.]+)').astype(float)
# Rating_Count & Review_Count
df['Rating_Count'] = df['Review Count'].astype(str).replace('[,]', '', regex=True)
df['Rating_Count'] = pd.to_numeric(df['Rating_Count'], errors='coerce').fillna(0).astype(int)
df['Review_Count'] = df['Rating_Count']  # if you only have one count column
# Platform (Amazon)
df['Platform'] = 'Amazon'

# STEP 2: FEATURE EXTRACTION FROM PRODUCT NAME
# Brand (first word before space)
df['Brand'] = df['Product Name'].str.split().str[0]
# Color (inside parentheses before first comma)
df['Color'] = df['Product Name'].str.extract(r'\(([^,]+),')
# RAM
df['RAM'] = df['Product Name'].str.extract(r'(\d+)\s*GB\s*RAM').astype(float)
# ROM
df['ROM'] = df['Product Name'].str.extract(r'(\d+)\s*GB\s*(?:ROM|Storage)').astype(float)
# Display Size (inches)
df['Display_Size'] = df['Product Name'].str.extract(r'(\d+\.?\d*)\s*inch').astype(float)
# Battery (mAh)
df['Battery'] = df['Product Name'].str.extract(r'(\d+)\s*mAh').astype(float)
# Front Camera MP
df['Front_Cam(MP)'] = df['Product Name'].str.extract(r'(\d+)\s*MP\s*Front').astype(float)
# Back Camera MP (biggest MP number in product name)
df['Back_Cam(MP)'] = df['Product Name'].apply(lambda x: max([int(n) for n in re.findall(r'(\d+)\s*MP', str(x))], default=None))
# Processor (words before 'Processor')
df['Processor'] = df['Product Name'].str.extract(r'([A-Za-z0-9\s\-]+Processor)')

In [95]:
df=df[['Product Name','Brand', 'Color', 'Platform', 'MRP', 'Selling_Price', 'Discount_Price','Discount', 'RAM', 'ROM', 'Display_Size', 'Battery', 
       'Front_Cam(MP)', 'Back_Cam(MP)', 'Processor','Rating', 'Rating_Count', 'Review_Count']]

In [97]:
df

,Product Name,Brand,Color,Platform,MRP,Selling_Price,Discount_Price,Discount,RAM,ROM,Display_Size,Battery,Front_Cam(MP),Back_Cam(MP),Processor,Rating,Rating_Count,Review_Count
0,"Samsung Galaxy M36 5G (Velvet Black, 6 GB RAM,...",Samsung,Velvet Black,Amazon,22999.0,17499.0,5500.0,24.0,6.0,128.0,NaN,NaN,NaN,50.0,NaN,3.8,0,0
1,"Samsung Galaxy M06 5G (Sage Green, 4GB RAM, 12...",Samsung,Sage Green,Amazon,13999.0,8749.0,5250.0,38.0,4.0,128.0,NaN,NaN,NaN,NaN,NaN,3.8,0,0
2,"Samsung Galaxy M05 (Mint Green, 4GB RAM, 64 GB...",Samsung,Mint Green,Amazon,9999.0,6249.0,3750.0,38.0,4.0,64.0,NaN,5000.0,NaN,50.0,NaN,4.0,5241,5241
3,"Redmi A4 5G (Starry Black, 4GB RAM, 128GB Stor...",Redmi,Starry Black,Amazon,11999.0,8798.0,3201.0,27.0,4.0,128.0,NaN,NaN,NaN,50.0,NaN,3.9,5052,5052
4,"POCO M7 5G, Ocean Blue (6GB, 128GB)",POCO,6GB,Amazon,12999.0,9299.0,3700.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,116,116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,"Samsung Galaxy M36 5G (Orange Haze, 6 GB RAM, ...",Samsung,Orange Haze,Amazon,22999.0,17499.0,5500.0,24.0,6.0,128.0,NaN,NaN,NaN,50.0,NaN,3.8,0,0
452,"Redmi Note 14 5G (Ivy Green, 8GB RAM 128GB Sto...",Redmi,Ivy Green,Amazon,22999.0,17999.0,5000.0,22.0,8.0,128.0,NaN,NaN,NaN,50.0,NaN,3.8,0,0
453,"Lava Bold N1 Pro (Stealth Black, 4 GB RAM, 128...",Lava,Stealth Black,Amazon,8399.0,6898.0,1501.0,18.0,4.0,128.0,NaN,5000.0,8.0,50.0,NaN,4.0,0,0
454,"Redmi 13 5G Prime Edition, Hawaiian Blue, 8GB+...",Redmi,NaN,Amazon,19999.0,11699.0,8300.0,42.0,NaN,NaN,NaN,NaN,NaN,108.0,NaN,4.0,0,0


In [99]:
df.to_csv("amazon_mobiles_features_for_edition.csv", index=False)

In [ ]:
# After manually typed the data

In [139]:
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')
# data loading
df=pd.read_excel("amazon_mobiles_500_V.xlsx")
df

,Product Name,Brand,Color,Platform,MRP,Selling_Price,Discount_Price,Discount,RAM,ROM,Display_Size,Battery,Front_Cam,Back_Cam,Processor,Rating,Rating_Count,Review_Count
0,"Samsung Galaxy M36 5G (Velvet Black, 6 GB RAM,...",Samsung,Velvet Black,Amazon,22999.0,17499.0,5500.0,24.0,6,128,"6.6"" FHD+ Super AMOLED, 120Hz","5000mAh, 25W",13MP,50MP OIS + 8MP + 2MP,Snapdragon 7s Gen 2,4.3★,2345,435
1,"Samsung Galaxy M06 5G (Sage Green, 4GB RAM, 12...",Samsung,Sage Green,Amazon,13999.0,8749.0,5250.0,38.0,4,128,"6.7"" HD+ LCD, 90Hz","5000mAh, 25W",8MP,50MP + 2MP,MediaTek Dimensity 6300,4.2★,1980,320
2,"Samsung Galaxy M05 (Mint Green, 4GB RAM, 64 GB...",Samsung,Mint Green,Amazon,9999.0,6249.0,3750.0,38.0,4,64,"6.7"" HD+ LCD, 90Hz","5000mAh, 25W",8MP,50MP + 2MP,Exynos 850,4.1★,1250,210
3,"Redmi A4 5G (Starry Black, 4GB RAM, 128GB Stor...",Redmi,Starry Black,Amazon,11999.0,8798.0,3201.0,27.0,4,128,"6.88"" FHD+ LCD, 120Hz","5160mAh, 18W",8MP,50MP + 2MP,Snapdragon 4 Gen 2,4.3★,3540,530
4,"POCO M7 5G, Ocean Blue (6GB, 128GB)",POCO,NaN,Amazon,12999.0,9299.0,3700.0,28.0,6,128,"6.6"" FHD+ LCD, 120Hz","5000mAh, 33W",13MP,50MP + 2MP,Dimensity 6080,4.2★,2150,415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,"Samsung Galaxy M36 5G (Orange Haze, 6 GB RAM, ...",Samsung,Orange Haze,Amazon,22999.0,17499.0,5500.0,24.0,6,128,6.6,5000,13,108,MediaTek G85*,4.4★,5201,950
452,"Redmi Note 14 5G (Ivy Green, 8GB RAM 128GB Sto...",Redmi,Ivy Green,Amazon,22999.0,17999.0,5000.0,22.0,8,128,6.8,5000,16,50,Snapdragon 4 Gen 2 AE,4.2★,1209,525
453,"Lava Bold N1 Pro (Stealth Black, 4 GB RAM, 128...",Lava,Stealth Black,Amazon,8399.0,6898.0,1501.0,18.0,4,128,6.8,4500,8,NaN,Snapdragon 4s Gen 2,4.3★,3024,770
454,"Redmi 13 5G Prime Edition, Hawaiian Blue, 8GB+...",Redmi,NaN,Amazon,19999.0,11699.0,8300.0,42.0,NaN,NaN,NaN,5000,8,50,NaN,4.3,4203,459


In [141]:
# Brand and Model (these are always strings)
df['Brand'] = df['Product Name'].str.split().str[0]
df['Brand_Model'] = df['Product Name'].apply(lambda x: ' '.join(str(x).split()[:3]))
# Clean Display_Size (keep only numbers and decimal)
df['Display_Size'] = df['Display_Size'].astype(str).str.replace(r'[^\d\.]', ' ', regex=True).str.strip()
# Clean Battery (keep only digits)
df['Battery'] = df['Battery'].astype(str).str.replace(r'[^\d]', '', regex=True)
# Clean Front Camera
df['Front_Cam'] = df['Front_Cam'].astype(str).str.replace(r'[^\d]', '', regex=True)
# Clean back camera and safely extract the first digit
df['Back_Cam'] = (df['Back_Cam'].astype(str).str.replace(r'[^\d]', ' ', regex=True).str.strip()                                
    .apply(lambda x: x.split()[0] if x.split() else pd.NA))
# Clean Rating (remove ★ )
df['Rating'] = df['Rating'].astype(str).str.replace('★', '', regex=False)

# Remove 'GB' from RAM and ROM
df['RAM'] = df['RAM'].astype(str).str.replace('GB', '', regex=False)

df['ROM'] = df['ROM'].astype(str)
df['ROM'] = df['ROM'].str.replace('1TB', '1000', regex=False)
df['ROM'] = df['ROM'].str.replace('GB', '', regex=False)

# Extract numeric display size (e.g., 6.5 from "6.5 inch AMOLED")
df['Display_Size'] = df['Display_Size'].astype(str).str.replace(r'[^\d\.]', '', regex=True)

# Extract only digits from Battery (e.g., 5000 from "5000 mAh", etc.)
df['Battery'] = df['Battery'].astype(str).str.replace(r'[^\d]', '', regex=True)

# Extract first number from Front_Cam and Back_Cam without converting to NaN
df['Front_Cam'] = df['Front_Cam'].astype(str).str.replace(r'[^\d]', '', regex=True).str[:3]
df['Back_Cam'] = df['Back_Cam'].astype(str).str.replace(r'[^\d]', '', regex=True).str[:3]


In [145]:
int_columns = ['Selling_Price','Review_Count', 'Rating_Count', 'RAM', 'ROM', 'Battery', 'Front_Cam', 'Back_Cam']
float_columns = ['MRP', 'Rating', 'Discount', 'Display_Size']
# Convert integer columns
for col in int_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')  # Nullable integer type
# Convert float columns
for col in float_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype(float)
# Final: Convert Discount_Price to float
df['Discount_Price'] = pd.to_numeric(df['Discount_Price'], errors='coerce')

In [149]:
df=df[df['Discount_Price'] >= 0]  # Keep only positive discounts
df

,Product Name,Brand,Color,Platform,MRP,Selling_Price,Discount_Price,Discount,RAM,ROM,Display_Size,Battery,Front_Cam,Back_Cam,Processor,Rating,Rating_Count,Review_Count,Brand_Model
0,"Samsung Galaxy M36 5G (Velvet Black, 6 GB RAM,...",Samsung,Velvet Black,Amazon,22999.0,17499,5500.0,24.0,6,128,6.6120,500025,13,50,Snapdragon 7s Gen 2,4.3,2345,435,Samsung Galaxy M36
1,"Samsung Galaxy M06 5G (Sage Green, 4GB RAM, 12...",Samsung,Sage Green,Amazon,13999.0,8749,5250.0,38.0,4,128,6.7900,500025,8,50,MediaTek Dimensity 6300,4.2,1980,320,Samsung Galaxy M06
2,"Samsung Galaxy M05 (Mint Green, 4GB RAM, 64 GB...",Samsung,Mint Green,Amazon,9999.0,6249,3750.0,38.0,4,64,6.7900,500025,8,50,Exynos 850,4.1,1250,210,Samsung Galaxy M05
3,"Redmi A4 5G (Starry Black, 4GB RAM, 128GB Stor...",Redmi,Starry Black,Amazon,11999.0,8798,3201.0,27.0,4,128,6.8812,516018,8,50,Snapdragon 4 Gen 2,4.3,3540,530,Redmi A4 5G
4,"POCO M7 5G, Ocean Blue (6GB, 128GB)",POCO,NaN,Amazon,12999.0,9299,3700.0,28.0,6,128,6.6120,500033,13,50,Dimensity 6080,4.2,2150,415,"POCO M7 5G,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,"Samsung Galaxy M36 5G (Orange Haze, 6 GB RAM, ...",Samsung,Orange Haze,Amazon,22999.0,17499,5500.0,24.0,6,128,6.6000,5000,13,108,MediaTek G85*,4.4,5201,950,Samsung Galaxy M36
452,"Redmi Note 14 5G (Ivy Green, 8GB RAM 128GB Sto...",Redmi,Ivy Green,Amazon,22999.0,17999,5000.0,22.0,8,128,6.8000,5000,16,50,Snapdragon 4 Gen 2 AE,4.2,1209,525,Redmi Note 14
453,"Lava Bold N1 Pro (Stealth Black, 4 GB RAM, 128...",Lava,Stealth Black,Amazon,8399.0,6898,1501.0,18.0,4,128,6.8000,4500,8,<NA>,Snapdragon 4s Gen 2,4.3,3024,770,Lava Bold N1
454,"Redmi 13 5G Prime Edition, Hawaiian Blue, 8GB+...",Redmi,NaN,Amazon,19999.0,11699,8300.0,42.0,<NA>,<NA>,NaN,5000,8,50,NaN,4.3,4203,459,Redmi 13 5G


#### Data Validation

In [152]:
df[df['Selling_Price'] > df['MRP']]

,Product Name,Brand,Color,Platform,MRP,Selling_Price,Discount_Price,Discount,RAM,ROM,Display_Size,Battery,Front_Cam,Back_Cam,Processor,Rating,Rating_Count,Review_Count,Brand_Model


In [ ]:
import numpy as np
df.loc[df['Discount_Price'] < 0, 'Discount_Price'] = np.nan  # or 0

In [154]:
df.columns

Index(['Product Name', 'Brand', 'Color', 'Platform', 'MRP', 'Selling_Price',
       'Discount_Price', 'Discount', 'RAM', 'ROM', 'Display_Size', 'Battery',
       'Front_Cam', 'Back_Cam', 'Processor', 'Rating', 'Rating_Count',
       'Review_Count', 'Brand_Model'],
      dtype='object')

In [62]:
df['Brand_Model'].unique()

array(['Samsung Galaxy M36', 'Samsung Galaxy M06', 'Samsung Galaxy M05',
       'Redmi A4 5G', 'POCO M7 5G,', 'OnePlus Nord CE4', 'POCO M6 Plus',
       'Samsung Galaxy M35', 'iQOO Z10 Lite', 'Redmi 13 5G',
       'Redmi 14C 5G', 'realme NARZO 80', 'Samsung Galaxy S24',
       'HONOR 200 5G', 'POCO C75 5G,', 'Lava Bold N1', 'iPhone 16 Plus',
       'Lava O3', 'Samsung Galaxy M16', 'Vivo V50 5G', 'Redmi Note 14',
       'Samsung Galaxy A55', 'iQOO Z10x 5G', 'Oppo K12x 5G',
       'OnePlus Nord 5', 'Samsung Galaxy A56', 'Samsung Galaxy A06',
       'iQOO Z9x 5G', 'Lava Storm Lite', 'Tecno POP 9',
       'Infinix Note 50s', 'OnePlus 13R', 'Acer Super ZX',
       'realme Motorola Edge', 'realme P1', 'Redmi A3X', 'iQOO 13 5G',
       'OnePlus Nord 4', 'Samsung Galaxy A36', 'Vivo Y39 5G', 'Xiaomi 14',
       'POCO M7 5G', 'Motorola Moto G85', 'realme NARZO 80x',
       'OPPO A5 Pro', 'realme NARZO N65', 'Nokia 220 4G', 'Redmi A5',
       'Samsung Galaxy Z', 'VIVO Y19 5G', 'Motorola G05 4G',


In [156]:
# Step 1: Define invalid values to remove
invalid_color_values = ['8GB+256GB', '4GB','6GB','Amber Gold 6GB RAM','Email Delivery']
# Replace them with NaN
df['Color'] = df['Color'].replace(invalid_color_values, pd.NA)
# Step 2: Fill NaN with the most common color (mode)
most_common_color = df['Color'].mode()[0]
df['Color'] = df['Color'].fillna(most_common_color)
# Step 3: Remove \u202f and extra spaces from color names
df['Color'] = df['Color'].str.replace('\u202f', ' ', regex=True).str.strip()

In [158]:
# Remove and strip whitespace
df['Brand_Model'] = df['Brand_Model'].str.replace(
    r'\||\+|\(Glossy|\(Midnight|\(Olive|\(White|\(Meteor|\(Metaverse|\(Majestic|\(3A\)|Speed|CIVI|Jaisalmer|Original 25W|Classic|CMF BY','',
    regex=True).str.strip()

# Step 1: Define invalid values to remove
invalid_color_values = ['Portronics Clamp M3', 'Elfora 360Â° Mobile' ,'Acko 1-Year Total', 'Ambrane Mobile Holding']
# Replace them with NaN
df['Brand_Model'] = df['Brand_Model'].replace(invalid_color_values, pd.NA)
# Step 2: Fill NaN with the most common color (mode)
most_common_color = df['Brand_Model'].mode()[0]
df['Brand_Model'] = df['Brand_Model'].fillna(most_common_color)
# Remove '*' symbol and strip whitespace
df['Processor'] = df['Processor'].str.replace('*', '', regex=False).str.strip()

In [160]:
# renamed those columns
df.rename(columns={
    'Front_Cam': 'Front_Cam(MP)',
    'Back_Cam': 'Back_Cam(MP)',
    'Rating':'Ratings'
}, inplace=True)

In [162]:
df.drop(columns=['Product Name'], inplace=True)

df=df[['Brand', 'Brand_Model', 'Color', 'Platform', 'MRP', 'Selling_Price', 'Discount_Price',
    'Discount', 'RAM', 'ROM', 'Display_Size', 'Battery', 
    'Front_Cam(MP)', 'Back_Cam(MP)', 'Processor','Ratings', 'Rating_Count', 'Review_Count'
    ]]

In [164]:
# final data set after extraction and cleaning and validation on column wise
df

,Brand,Brand_Model,Color,Platform,MRP,Selling_Price,Discount_Price,Discount,RAM,ROM,Display_Size,Battery,Front_Cam(MP),Back_Cam(MP),Processor,Ratings,Rating_Count,Review_Count
0,Samsung,Samsung Galaxy M36,Velvet Black,Amazon,22999.0,17499,5500.0,24.0,6,128,6.6120,500025,13,50,Snapdragon 7s Gen 2,4.3,2345,435
1,Samsung,Samsung Galaxy M06,Sage Green,Amazon,13999.0,8749,5250.0,38.0,4,128,6.7900,500025,8,50,MediaTek Dimensity 6300,4.2,1980,320
2,Samsung,Samsung Galaxy M05,Mint Green,Amazon,9999.0,6249,3750.0,38.0,4,64,6.7900,500025,8,50,Exynos 850,4.1,1250,210
3,Redmi,Redmi A4 5G,Starry Black,Amazon,11999.0,8798,3201.0,27.0,4,128,6.8812,516018,8,50,Snapdragon 4 Gen 2,4.3,3540,530
4,POCO,"POCO M7 5G,",Starry Black,Amazon,12999.0,9299,3700.0,28.0,6,128,6.6120,500033,13,50,Dimensity 6080,4.2,2150,415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,Samsung,Samsung Galaxy M36,Orange Haze,Amazon,22999.0,17499,5500.0,24.0,6,128,6.6000,5000,13,108,MediaTek G85,4.4,5201,950
452,Redmi,Redmi Note 14,Ivy Green,Amazon,22999.0,17999,5000.0,22.0,8,128,6.8000,5000,16,50,Snapdragon 4 Gen 2 AE,4.2,1209,525
453,Lava,Lava Bold N1,Stealth Black,Amazon,8399.0,6898,1501.0,18.0,4,128,6.8000,4500,8,<NA>,Snapdragon 4s Gen 2,4.3,3024,770
454,Redmi,Redmi 13 5G,Starry Black,Amazon,19999.0,11699,8300.0,42.0,<NA>,<NA>,NaN,5000,8,50,NaN,4.3,4203,459


In [168]:
# Saving after cleaning file (Amazon)
df.to_csv("Amazon_Final.csv", index=False)

#### deoping nan values rows becasues those are keypad mobiles

In [6]:
import pandas as pd
df=pd.read_csv('Amazon_Final.csv')

In [8]:
df1=df.dropna()

In [10]:
df1.to_csv('Amazon_Final.csv',index=False)